In [11]:
#!pip install transformers

In [1]:
import pandas as pd

csv_path = "/home/alex/dev/AiLearning/DataSetStore/twitter_msg_sentiment_anal/train.csv"

df = pd.read_csv(csv_path)
df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [2]:

# https://huggingface.co/docs/transformers/model_doc/funnel

from transformers import AutoTokenizer, FunnelBaseModel
import torch

tokenizer = AutoTokenizer.from_pretrained("funnel-transformer/small-base")
model = FunnelBaseModel.from_pretrained("funnel-transformer/small-base")
inputs = tokenizer("Hello, my dog is cute cccccc", return_tensors="pt")
outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state
print (last_hidden_states)

tensor([[[-0.0237, -0.0414, -0.0016,  ...,  0.0363,  0.0031, -0.0376],
         [ 0.2734, -0.0644, -0.1587,  ...,  0.1573,  0.1821,  0.1352],
         [ 0.2198,  0.1701, -0.0584,  ...,  0.0731,  0.0245,  0.0226]]],
       grad_fn=<NativeLayerNormBackward0>)


In [3]:

tokenizer = AutoTokenizer.from_pretrained("funnel-transformer/small-base")
model = FunnelBaseModel.from_pretrained("funnel-transformer/small-base")

def funnel_encoder (txt: str, verbose=True, flatten=True):

    with torch.no_grad():
        inputs = tokenizer(txt, return_tensors="pt")
        outputs = model(**inputs)
        result = outputs.last_hidden_state.numpy()

    if flatten:
        shape = result.shape
        result = result.reshape(shape[0]*shape[1]*shape[2])
    
    if verbose:
        print ("Predicting: {} {}".format(txt,result))
    return result

In [32]:
INPUT_SIZE = funnel_encoder(txt="Hellow world  ccc").shape[0]
INPUT_SIZE

Predicting: Hellow world  ccc [-0.0286085  -0.03808539  0.00674396 ...  0.23151398 -0.06406398
 -0.07982317]


1536

In [ ]:
from tqdm import tqdm
import numpy as np

Y = []
X = []
for (twt,lab) in tqdm(zip(df['tweet'],df['label'])):
    x = funnel_encoder(txt=twt,verbose=False)
    if x.shape == INPUT_SIZE:
        X.append(x)
        Y.append(lab)
X = np.array(X)
Y = np.array(Y)

print (X.shape,Y.shape)

10480it [05:15, 33.29it/s]

In [22]:

class MyDataset(torch.utils.data.Dataset):
    
    def __init__(self, text_list, label_list):
        self.text_list = text_list
        self.label_list = label_list
    
    def __len__(self):
        return len(self.text_list)
    
    def __getitem__(self, index):
        return self.text_list[index],self.label_list[index]


In [23]:
import gc
from torch import nn
from torch import optim
import torch.nn.functional as F

def execute (runnable):
    if torch.cuda.device_count()>0:
        print ("  --> Let's work with cuda")
        device = torch.device("cuda:0")
        gc.collect()
        torch.cuda.empty_cache()
        if runnable is not None:
            with torch.cuda.device(0):
                return runnable(device)
        return device
    else:
        print ("  --> Let's work with CPU")
        device = torch.device("cpu")
        if runnable is not None:
            return runnable(device)
        return device

for x in range (3):
    execute(None)

def to_device(data, device):
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    print ("  --> Copied data: ",data.shape)
    return data.to(device, non_blocking=True)

def load_to_device(loader,device):
    load_list=[]
    for i, data in enumerate(loader):
        try:
            load_list.append((to_device(data[0],device), to_device(data[1],device)))
        except Exception as e:
            print (e)
    return load_list

BATCH_SIZE = 1000

train_dataset = MyDataset(text_list=X,label_list=Y)

# for (i,(x,y)) in enumerate(train_dataset):
#     if i%100==0:
#         print (x,y)

train_set_loader = torch.utils.data.DataLoader(
        train_dataset,batch_size=BATCH_SIZE,shuffle=True)

train_list = load_to_device(train_set_loader,execute(None))


  --> Let's work with cuda
  --> Let's work with cuda
  --> Let's work with cuda
  --> Let's work with cuda
  --> Copied data:  torch.Size([857, 1536])
  --> Copied data:  torch.Size([857])


In [28]:

class Perceptron(nn.Module):
    
    def __init__(self, input_size, hidden_size, output_size, hidden_layer_quantity, batch_norm_enabled=True):
        
        super().__init__()
        self.layers = []
        
        # Making the basic perceptron:---------------------------------
        dropout_base = 0.05
        self.layers.append(nn.Linear(input_size,hidden_size))
        self.layers.append(nn.Dropout( dropout_base ))
        
        for cnt in range(0,hidden_layer_quantity):
            self.layers.append(nn.Linear(hidden_size,hidden_size))
            if batch_norm_enabled == True:
                self.layers.append(nn.BatchNorm1d(hidden_size))
            drop_coef = (1.0/hidden_layer_quantity) * cnt + 2*dropout_base
            drop_coef = 0.80 if drop_coef>0.80 else drop_coef
            self.layers.append(nn.Dropout( drop_coef ))
        self.layers.append(nn.Linear(hidden_size,output_size))

        # Registering modules for torch:-------------------------------
        for cnt,layer in enumerate(self.layers):
            key = "layer{}".format(cnt)
            self.add_module(key,layer)

    def forward(self, xtr):
        for cnt,layer in enumerate(self.layers):
            if cnt<len(self.layers)-1:
                xtr = F.leaky_relu(layer(xtr))
            else :
                xtr = F.sigmoid(layer(xtr))
        return xtr

In [ ]:

perceptron = Perceptron(INPUT_SIZE,100,1,2).to(execute(None))

for idx, module in enumerate(perceptron.modules()):
     print(idx, '->', module)

criteria = nn.MSELoss()
optimizer = optim.SGD(perceptron.parameters(), lr=0.01, momentum=0.05)

In [19]:
df_cp = df.copy()

def predict (txt: str, verbose=True):
    input_ids = torch.tensor(tokenizer.encode(txt, add_special_tokens=True)).unsqueeze(0)
    outputs = model(input_ids)
    result = outputs.sop_logits
    
    if verbose:
        print ("Predicting: {} {} {}".format(txt,input_ids,result))
    return result

    


In [20]:

def adapt_perceptron (device):
    epoch_max_num = 5000
    history=[]
    for epoch_num in range(epoch_max_num):
        running_loss = 0.0
        for train_batch in train_list:
            x_train, y_train = train_batch[0], train_batch[1]
            #y_train[y_train==1.0]=0.75
            optimizer.zero_grad()
            y_pred = perceptron(x_train)
            loss = criteria(y_pred, y_train)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        history.append((epoch_num,running_loss))
        if epoch_num % 100==0 :
            print("Epoch [{}/{}] loss: {}".format(epoch_num,epoch_max_num,running_loss))

    return history

train_history = execute(runnable=adapt_perceptron)


Predicting:  @user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run tensor([[    2,    13,     1, 16704,    76,    21,   321,    25, 22880,   192,
            17,    25,    86, 14716,    24,  5501,    18,    33,  2627,    77,
            33, 22880,     9,  6926,  3169,     3]]) tensor([[ 0.3356, -0.3219]], grad_fn=<AddmmBackward0>)
Predicting: @user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx.    #disapointed #getthanked tensor([[    2,    13,     1, 16704,    13,     1, 16704,  3669,    26,  6926,
           102,  3072,  3251,    31,    92,    22,    38,   275,  1679,    59,
           221,    22,    38,  1994, 14013,  1019,    18,    19,    13,  9251,
           396,     9,  6926,  2906,    58,  3132,    69,  6926,  3060,  5904,
            69,     3]]) tensor([[-0.0783, -0.4551]], grad_fn=<AddmmBackward0>)
Predicting:   bihday your majesty tensor([[    2,  1732,   252,  1185,   154, 10281,    

0     [[tensor(0.3356, grad_fn=<UnbindBackward0>), t...
1     [[tensor(-0.0783, grad_fn=<UnbindBackward0>), ...
2     [[tensor(0.1546, grad_fn=<UnbindBackward0>), t...
3     [[tensor(0.1967, grad_fn=<UnbindBackward0>), t...
4     [[tensor(0.1497, grad_fn=<UnbindBackward0>), t...
                            ...                        
95    [[tensor(0.2577, grad_fn=<UnbindBackward0>), t...
96    [[tensor(0.1967, grad_fn=<UnbindBackward0>), t...
97    [[tensor(0.0438, grad_fn=<UnbindBackward0>), t...
98    [[tensor(0.1680, grad_fn=<UnbindBackward0>), t...
99    [[tensor(-0.0478, grad_fn=<UnbindBackward0>), ...
Name: tweet, Length: 100, dtype: object